# NBA 2024-25: Slump Shots & Recovery Shots
## Notebook 01: Data Acquisition
This notebook handles the acquisition of raw, shot-level NBA data for the 2024-25 regular season (via the NBA API). Detailed play-by-play and shot location data is pulled for all players, creating a complete dataset that will serve as the foundation for all downstream analysis.

In [2]:
# Install NBA API
!pip install nba_api

In [3]:
# Import libraries
import pandas as pd
import time
from tqdm import tqdm
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.static import teams

In [4]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [5]:
# Display options
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
pd.set_option("display.width", 160)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.float_format", lambda x: f"{x:.2f}")

---
## Acquire Shot-Level Data
#### (for all 30 teams)

In [7]:
# Get team IDs
nba_teams = teams.get_teams()
team_ids = [team["id"] for team in nba_teams]

print(f"Number of teams: {len(team_ids)}")

Number of teams: 30


In [ ]:
# --- Loop through all 30 teams and pull all shot data from the 2024-25 regular season ---
all_shots = []

for team_id in tqdm(team_ids, desc="Downloading shots"):

    shot_data = shotchartdetail.ShotChartDetail(
        team_id=team_id,
        player_id=0,   # 0 = all players on that team
        season_nullable="2024-25",
        season_type_all_star="Regular Season",
        context_measure_simple="FGA"
    )

    df = shot_data.get_data_frames()[0]
    all_shots.append(df)
    time.sleep(1.5)

In [ ]:
# Combine into one DataFrame
shots_2024_25 = pd.concat(all_shots, ignore_index=True)

print(f"{shots_2024_25.shape[0]:,} rows")

---
## Save

In [ ]:
# Inspect structure
shots_2024_25.head(10)

In [ ]:
# Inspect columns
shots_2024_25.columns

In [ ]:
# Save to parquet
shots_2024_25.to_parquet("nba_2024_25_shot_level_data_raw.parquet")